## Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import RandomizedSearchCV

## Puxando Dataframe

In [ ]:
%run Pre_Processamento.ipynb

In [ ]:
%store -r df_1

In [ ]:
df = df_1

## Preparação dos Dados


Nesta etapa, os dados foram divididos em duas versões distintas para testar e comparar os resultados do modelo sob diferentes condições:

##### Remoção de Outliers (``df1``):

A partir do método IQR (Interquartile Range), outliers são identificados e removidos do dataset. Esta medida de dispersão estatística proporciona a amplitude interquartil, facilitando a detecção e filtragem de valores extremos. A aplicação desse método resultou em uma redução aproximada de 9% nos registros originais.

In [ ]:
q1 = df['items_sold'].quantile(0.25)
q3 = df['items_sold'].quantile(0.75)

iqr = q3 - q1

lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

df1 = df[(df['items_sold'] >= lower_bound) & (df['items_sold'] <= upper_bound)]

##### Retirada da Black Friday (``df2``):

Nesta estratégia, semanas relacionadas à Black Friday e início de dezembro são excluídas, dado que tradicionalmente apresentam padrões atípicos de vendas.

In [ ]:
df2 = df.drop(df[df['semana_do_ano'].isin([46, 47, 48, 49])].index)

##### Divisão dos datasets

A seguir, foram preparados os conjuntos de dados para treinamento e teste. Em ambos os dataests foram removidas colunas irrelevantes para a modelagem e foram separadas as features (``X``) dos targets (``y``), para cada dataset.

In [ ]:
X1 = df1.drop(['revenue', 'items_sold', 'date', 'semana_do_ano', 'trimestre', 'dia_do_ano', 'dia_da_semana',
       'fim_de_semana'], axis=1)
y1 = df1['items_sold']

In [ ]:
X2 = df2.drop(['revenue', 'items_sold', 'date', 'semana_do_ano', 'trimestre', 'dia_do_ano', 'dia_da_semana',
       'fim_de_semana'], axis=1)
y2 = df2['items_sold']

Após o tratamento inicial dos dados, os conjuntos de features (``X``) e target (``y``) foram preparados para os dois dataframes. Os atributos relacionados a 'revenue', 'items_sold', 'date' e alguns outros foram removidos, pois eles não seriam úteis na etapa de treinamento do modelo.

Posteriormente, os dados foram divididos em conjuntos de treino e teste utilizando uma proporção de 70% para treino e 30% para teste.

In [ ]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.3, random_state=42)

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

## Random search

##### Ajuste dos Hiperparâmetros usando Random Search
>O método ``Random Search`` é utilizado na otimização de hiperparâmetros. Embora seja possível abranger um vasto espaço de hiperparâmetros, por uma questão de viabilidade de capacidade computacional, a busca foi restrita ao seguinte intervalo.

In [ ]:
random_param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 10],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rf_model = RandomForestRegressor()

> O modelo ideal para a utilização do `Random Search` seria igual a :
~~~python
random_param_grid = {
    'n_estimators': np.arange(50, 1500, 50),
    'max_depth': [None] + list(np.arange(5, 31)),
    'min_samples_split': np.arange(2, 11),
    'min_samples_leaf': np.arange(1, 11),
}
~~~

Aqué feita a busca pelos melhores hiperparâmetros para a versão 1

In [ ]:
random_search1 = RandomizedSearchCV(
    rf_model, random_param_grid, n_iter=5, scoring='neg_mean_squared_error', cv=5, random_state=42
)
random_search1.fit(X1_train, y1_train)

best_random_params1 = random_search1.best_params_

Aqué feita a busca pelos melhores hiperparâmetros para a versão 2

In [ ]:
random_search2 = RandomizedSearchCV(
    rf_model, random_param_grid, n_iter=5, scoring='neg_mean_squared_error', cv=5, random_state=42
)
random_search2.fit(X2_train, y2_train)

best_random_params2 = random_search2.best_params_

>O resultado encontrado pelo `Random Search foram` para o modelo de `RandomForestRegressor`
~~~python
{
 'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_depth': None
}
~~~

## Modelo Random Forest

### Treinamento sem Outliers

In [ ]:
rf_model1 = RandomForestRegressor(n_estimators= 100, min_samples_split= 2, min_samples_leaf= 2, max_depth= None)
rf_model1.fit(X1_train, y1_train)

In [ ]:
y1_pred = rf_model1.predict(X1_test)

In [ ]:
mae1 = mean_absolute_error(y1_test, y1_pred)
print(f"Erro Médio Absoluto (MAE): {mae1}")

mse1 = mean_squared_error(y1_test, y1_pred)
print(f"Erro Quadrático Médio (MSE): {mse1}")

rmse1 = np.sqrt(mse1)
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse1}")

r21 = r2_score(y1_test, y1_pred)
print(f"Coeficiente de Determinação (R²): {r21}")

Os resultados obtidos são:

- Erro Médio Absoluto (MAE): 0.9307597799264964
- Erro Quadrático Médio (MSE): 1.733232060134655
- Raiz do Erro Quadrático Médio (RMSE): 1.316522715388783
- Coeficiente de Determinação (R²): 0.45169788386178256

| Métrica                               | Valor                  |
|---------------------------------------|------------------------|
| Erro Médio Absoluto (MAE)             | 0.9307597799264964     |
| Erro Quadrático Médio (MSE)           | 1.733232060134655      |
| Raiz do Erro Quadrático Médio (RMSE)  | 1.316522715388783      |
| Coeficiente de Determinação (R²)      | 0.45169788386178256    |

### Treinamento sem Semanas da Black Friday

Nesta fase, o modelo é treinado com os dados onde semanas específicas foram excluídas.

In [ ]:
rf_model2 = RandomForestRegressor(n_estimators= 100, min_samples_split= 2, min_samples_leaf= 2, max_depth= None)
rf_model2.fit(X2_train, y2_train)

In [ ]:
y2_pred = rf_model2.predict(X2_test)

In [ ]:
mae2 = mean_absolute_error(y2_test, y2_pred)
print(f"Erro Médio Absoluto (MAE): {mae2}")

mse2 = mean_squared_error(y2_test, y2_pred)
print(f"Erro Quadrático Médio (MSE): {mse2}")

rmse2 = np.sqrt(mse2)
print(f"Raiz do Erro Quadrático Médio (RMSE): {rmse2}")

r22 = r2_score(y2_test, y2_pred)
print(f"Coeficiente de Determinação (R²): {r22}")

| Métrica                               | Valor                  |
|---------------------------------------|------------------------|
| Erro Médio Absoluto (MAE)             | 1.4488690788721887     |
| Erro Quadrático Médio (MSE)           | 8.389110146760984      |
| Raiz do Erro Quadrático Médio (RMSE)  | 2.8963960617914437     |
| Coeficiente de Determinação (R²)      | 0.7167009979625879     |

## Discussão dos Resultados

Ao analisar os resultados, notamos as seguintes observações:

**Retirada de Outliers**: A abordagem de remoção de outliers apresentou um erro (tanto MAE quanto RMSE) menor, indicando que o modelo teve um desempenho melhor em termos de precisão da previsão. Entretanto, o coeficiente de determinação (R²) é mais baixo, sugerindo que, apesar das previsões estarem mais próximas dos valores reais, o modelo pode não estar capturando toda a variabilidade dos dados. Uma possível explicação para isso é que ao retirar os outliers, o modelo perdeu parte da informação contida nos dados extremos, o que pode limitar sua capacidade de generalização em certas situações.

**Retirada da Black-Friday**: Por outro lado, ao remover as semanas associadas à Black Friday, notamos que o erro é mais elevado, indicando que o modelo tem menos precisão em suas previsões. Entretanto, o coeficiente de determinação (R²) é significativamente maior, sugerindo que o modelo está capturando uma maior proporção da variabilidade dos dados. Isso indica que, apesar de cometer erros maiores em algumas previsões, o modelo é capaz de adaptar-se a uma variedade maior de situações, possivelmente devido ao fato de que ele está lidando com mais dados.

# Considerações finais

Após uma avaliação dos modelos candidatos (Random Forest, XGBoost e Prophet), concluímos que, comparado aos outros modelos, o Random Forest não se destacou nas métricas de avaliação estabelecidas para nosso projeto. Portanto, optamos por não selecionar o Random Forest como a solução final.